In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
torch.set_num_threads(8)

# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            # 'dataset': "CIFAR100",
            'dataset': "BiasedMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 5,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 20,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 1,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.BCEWithLogitsLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.000,
            'lambda': .0,
            'lambda_old': 0.0,
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=BiasedMNIST/seed=10_epoch=5_lr=0.001_alpha=0.0_tau=1


In [2]:
from datasets import BiasedMNIST

if  params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = params['per_task_examples'],
                                random_class_idx=True)
    input_dim = (3, 28, 28)
    class_idx = benchmark.class_idx
    num_classes = len(class_idx)

[8 2 5 6 3 1 0 7 4 9]


In [3]:
from algorithms.icarl import iCaRL
from metrics import FairMetricCollector
from backbones import MLP2Layers2

backbone = MLP2Layers2(
    input_dim=input_dim, 
    hidden_dim_1=256, 
    hidden_dim_2=256, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])

# from backbones import ResNet18
# backbone = ResNet18(
#     input_dim=input_dim, 
#     output_dim=num_classes,
#     class_idx=class_idx,
#     config=params
#     ).to(params['device'])

algorithm = iCaRL(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = FairMetricCollector(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])


iCaRL


In [4]:
metric_manager_callback.meters['accuracy'].data.shape

(6, 6, 5)

In [5]:
from trainers.baselines import BaseContinualTrainer as ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [6]:
# CIFAR10: self.dataset.data.shape=(50000, 32, 32, 3)


In [7]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 0.8008870159018193, 'loss': 0.0, 'std': 0.007252518981901546, 'EER': -1, 'EO': [0.37594035691394256, 0.34714210014017977], 'DP': -1, 'accuracy_s0': 0.9828791478769658, 'accuracy_s1': 0.6213379193499047, 'classwise_accuracy': {8: array([773, 974]), 2: array([ 834, 1032])}, 'DP_ingredients': {'class_pred_count_s0': {8: 489, 2: 506}, 'class_pred_count_s1': {8: 482, 2: 529}, 'class_pred_count': {8: 971, 2: 1035}, 'count_s0': 995, 'count_s1': 1011, 'count': 2006}}
[2] Eval metrics for task 1 >> {'accuracy': 0.7882373891727552, 'loss': 0.0, 'std': 0.06544478136988041, 'EER': -1, 'EO': [0.5478443477380832, 0.2738089421880814], 'DP': -1, 'accuracy_s0': 0.9939691121563303, 'accuracy_s1': 0.583142467193248, 'classwise_accuracy': {8: array([704, 974]), 2: array([ 881, 1032])}, 'DP_ingredients': {'class_pred_count_s0': {8: 492, 2: 503}, 'class_pred_count_s1': {8: 363, 2: 648}, 'class_pred_count'

In [8]:
metric_manager_callback.meters['accuracy'].get_data()

array([[0.815, 0.   , 0.   , 0.   , 0.   ],
       [0.676, 0.604, 0.   , 0.   , 0.   ],
       [0.61 , 0.574, 0.486, 0.   , 0.   ],
       [0.56 , 0.519, 0.486, 0.537, 0.   ],
       [0.519, 0.515, 0.486, 0.528, 0.508]])

In [ ]:
np.mean(metric_manager_callback.meters['accuracy'].compute_overall())

[0.03727138150040593,
 0.03517587226968666,
 0.07032850022082138,
 0.08288781294947618,
 0.09570917188363709]

In [ ]:
[np.round(x, 3) for x in metric_manager_callback.meters['EO'].compute_overall()]

In [ ]:
np.mean(metric_manager_callback.meters['EO'].compute_overall())

[0.0,
 0.0029377586790896304,
 0.02922144834681062,
 0.04689474638930541,
 0.056992683670983445]

In [ ]:
[np.round(x, 3) for x in metric_manager_callback.meters['DP'].compute_overall()]

In [ ]:
np.mean(metric_manager_callback.meters['DP'].compute_overall())